In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
#from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [ ]:
df = pd.read_csv('amharic.csv', delimiter=',')
df.head()

,id,tweet,subtask_a
0,0,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...,NOT
1,1,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደረ...,NOT
2,2,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...,NOT
3,3,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...,NOT
4,4,ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...,OFF


In [ ]:
text = df.tweet
category = df.subtask_a
le = LabelEncoder()
category = le.fit_transform(category)
category = category.reshape(-1,1)

In [ ]:
text_train,text_test,cat_train,cat_test = train_test_split(text,category,test_size=0.15)


In [ ]:
len([c for c in cat_train if c == 1]) / len([c for c in cat_train])

0.506313725490196

In [ ]:
max_words = 5000
max_len = 200
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(text_train)
sequences = tok.texts_to_sequences(text_train)
sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(32)(layer)
    layer = Dense(512)(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [ ]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(), metrics=["accuracy"])

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 200)]             0         
_________________________________________________________________
embedding_12 (Embedding)     (None, 200, 50)           250000    
_________________________________________________________________
lstm_12 (LSTM)               (None, 32)                10624     
_________________________________________________________________
dense_2 (Dense)              (None, 512)               16896     
_________________________________________________________________
activation_24 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 513

In [ ]:
model.fit(sequences_matrix,cat_train, batch_size=128, epochs=10, validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.00001)])

Epoch 1/10
160/160 [==============================] - 28s 160ms/step - loss: 0.6844 - accuracy: 0.5387 - val_loss: 0.6354 - val_accuracy: 0.6357


In [ ]:
test_sequences = tok.texts_to_sequences(text_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences, maxlen=max_len)

In [ ]:
accr = model.evaluate(test_sequences_matrix, cat_test)

141/141 [==============================] - 3s 20ms/step - loss: 0.6372 - accuracy: 0.6296


In [ ]:
predicts = model.predict(test_sequences_matrix)
pred_cat = (np.asarray(predicts)).round()

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

# Print f1, precision, and recall scores
print(precision_score(cat_test, pred_cat , average="macro"))
print(recall_score(cat_test, pred_cat , average="macro"))
print(f1_score(cat_test, pred_cat , average="macro"))

0.6362145390070921
0.6275070730237965
0.6226904027759202


0.6436838457206394

0.6257910384433276

0.6136171655179474